**Name: Tommaso Guasti**



# FINAL PROJECT






In [1]:
#r "nuget:FSharp.Data,5.*"
#r "nuget: FSharp.Stats"
#r "nuget: Plotly.NET, 3.*"
#r "nuget: Plotly.NET.Interactive, 3.*"

open System
open FSharp.Data
open Plotly.NET
open FSharp.Stats


Installed Packages FSharp.Data, 5.0.2 FSharp.Stats, 0.4.11 Plotly.NET, 3.0.1 Plotly.NET.Interactive, 3.0.2

Loading extensions from `C:\Users\user\.nuget\packages\plotly.net.interactive\3.0.2\interactive-extensions\dotnet\Plotly.NET.Interactive.dll`

In [2]:
#r "nuget: Accord"
#r "nuget: Accord.Statistics"


Installed Packages Accord, 3.8.0 Accord.Statistics, 3.8.0

In [3]:

open Accord
open Accord.Statistics.Models.Regression.Linear

LOAD DATA



In [4]:
let [<Literal>] ResolutionFolder = __SOURCE_DIRECTORY__
Environment.CurrentDirectory <- ResolutionFolder

In [5]:
#r "nuget:NovaSBE.Finance, 0.3.*-*"
open NovaSBE.Finance
open NovaSBE.Finance.Portfolio

Installed Packages NovaSBE.Finance, 0.3.0-beta.2

In [6]:
let [<Literal>] IdAndReturnsFilePath = "id_and_return_data.csv"
let [<Literal>] MySignalFilePath = "eqnpo_12m.csv"
let strategyName = "FirstStrategy"



Test


In [7]:
IO.File.ReadLines(IdAndReturnsFilePath) |> Seq.truncate 5


index,value
0,"id(string),eom(date),source(string),sizeGrp(string),obsMain(bool),exchMain(bool),primarySec(bool),gvkey(string),iid(string),permno(int Option),permco(int Option),excntry(string),curcd(string),fx(string),common(bool),compTpci(string),crspShrcd(int Option),compExchg(string),crsp_exchcd(int Option),adjfct(float Option),shares(float Option),gics(int Option),sic(int Option),naics(int Option),ff49(int Option),ret(float Option),retExc(float Option),prc(float Option),marketEquity(float Option)"
1,"crsp_86432,2000-01-31T00:00:00.0000000,CRSP,micro,1,1,true,115876,01,86432,16313,USA,USD,1,true,,11,,3,2,5.218,40101010,6020,522110,45,-0.003906,-0.00824925,15.9375,83.161875"
2,"crsp_85640,2000-01-31T00:00:00.0000000,CRSP,small,1,1,true,002193,01,85640,20300,USA,USD,1,true,,11,,1,1,102.496,35102020,8051,623110,11,-0.157143,-0.161485863,3.6875,377.954"
3,"crsp_86430,2000-01-31T00:00:00.0000000,CRSP,micro,1,1,true,115946,01,86430,16319,USA,USD,1,true,,11,,3,1,10.764,45103010,7372,511210,36,0.285714,0.2813712985,3.9375,42.38325"
4,"crsp_85756,2000-01-31T00:00:00.0000000,CRSP,nano,1,1,true,066389,01,85756,15930,USA,USD,1,true,,11,,3,1,1.582,,6720,,48,0.190685,0.1863418699,10.25,16.2155"


In [8]:
IO.File.ReadLines(MySignalFilePath) |> Seq.truncate 5


index,value
0,"id(string),eom(date),signal(float option)"
1,"comp_001034_01,2008-12-31T00:00:00.0000000,0.042924262"
2,"comp_001043_01,2000-01-31T00:00:00.0000000,-0.005633818"
3,"comp_001076_02,2010-12-31T00:00:00.0000000,0.0067473148"
4,"comp_001081_01,2007-10-31T00:00:00.0000000,0.035213376"


In [9]:
type IdAndReturnsType = 
    CsvProvider<Sample=IdAndReturnsFilePath,
                ResolutionFolder=ResolutionFolder>

type MySignalType = 
    CsvProvider<MySignalFilePath,
                ResolutionFolder=ResolutionFolder>


Read the data

In [10]:
let idAndReturnsCsv = IdAndReturnsType.GetSample()

let mySignalCsv = MySignalType.GetSample()


Another test for headers


In [11]:
idAndReturnsCsv.Headers


Value
"[ id(string), eom(date), source(string), sizeGrp(string), obsMain(bool), exchMain(bool), primarySec(bool), gvkey(string), iid(string), permno(int Option), permco(int Option), excntry(string), curcd(string), fx(string), common(bool), compTpci(string), crspShrcd(int Option), compExchg(string), crsp_exchcd(int Option), adjfct(float Option) ... (9 more) ]"


In [12]:
mySignalCsv.Headers



Value
"[ id(string), eom(date), signal(float option) ]"


In [13]:
let idAndReturnsRows = idAndReturnsCsv.Rows |> Seq.toList
let mySignalRows = mySignalCsv.Rows |> Seq.toList

In [14]:
let msfBySecurityIdAndMonth =
    idAndReturnsRows
    |> List.map(fun row -> 
        let id = Other row.Id
        let month = DateTime(row.Eom.Year,row.Eom.Month,1)
        let key = id, month
        key, row)
    |> Map    

let signalBySecurityIdAndMonth =
    mySignalRows
    |> List.choose(fun row -> 
        match row.Signal with
        | None -> None 
        | Some signal ->
            let id = Other row.Id
            let month = DateTime(row.Eom.Year,row.Eom.Month,1)
            let key = id, month
            Some (key, signal))
    |> Map    


In [15]:
let securitiesByFormationMonth =
    idAndReturnsRows
    |> List.groupBy(fun x -> DateTime(x.Eom.Year, x.Eom.Month,1))
    |> List.map(fun (ym, obsThisMonth) -> 
        let idsThisMonth = [ for x in obsThisMonth do Other x.Id ]
        ym, idsThisMonth)
    |> Map

let getInvestmentUniverse formationMonth =
    match Map.tryFind formationMonth securitiesByFormationMonth with
    | Some securities -> 
        { FormationMonth = formationMonth 
          Securities = securities }
    | None -> failwith $"{formationMonth} is not in the date range"
   


In [16]:
let testUniverseObs = getInvestmentUniverse (DateTime(2015,4,1))

In [17]:
let getMySignal (securityId, formationMonth) =
    match Map.tryFind (securityId, formationMonth) signalBySecurityIdAndMonth with
    | None -> None
    | Some signal ->
        Some { SecurityId = securityId 
               // if a high signal means low returns,
               // use `-signal` here instead of `signal`
               Signal = signal }


In [18]:
[ for securityId in testUniverseObs.Securities[0..4] do
    let testObs = (securityId, testUniverseObs.FormationMonth)
    getMySignal testObs ]


index Value 0 SecurityId Signal Other
 Item: crsp_79903 0.0586796141 1 SecurityId Signal Other
 Item: crsp_86008 0.025054875 2 SecurityId Signal Other
 Item: crsp_14176 -0.042204563 3 SecurityId Signal Other
 Item: crsp_90314 -0.039572466 4 SecurityId Signal Other
 Item: crsp_65008 -0.283430997

In [19]:
let getMySignals (investmentUniverse: InvestmentUniverse) =
    let listOfSecuritySignals =
        investmentUniverse.Securities
        |> List.choose(fun security -> 
            getMySignal (security, investmentUniverse.FormationMonth))    
    
    { FormationMonth = investmentUniverse.FormationMonth 
      Signals = listOfSecuritySignals }


In [20]:
getMySignals testUniverseObs

FormationMonth,Signals
2015-04-01 00:00:00Z,FSharpList<SecuritySignal> - SecurityId: Other Item: crsp_79903 Signal: 0.0586796141 - SecurityId: Other Item: crsp_86008 Signal: 0.025054875 - SecurityId: Other Item: crsp_14176 Signal: -0.042204563 - SecurityId: Other Item: crsp_90314 Signal: -0.039572466 - SecurityId: Other Item: crsp_65008 Signal: -0.283430997 - SecurityId: Other Item: crsp_20750 Signal: 0.0253211299 - SecurityId: Other Item: crsp_12622 Signal: 0.056201798 - SecurityId: Other Item: crsp_77462 Signal: 0.0974887048 - SecurityId: Other Item: crsp_66617 Signal: -0.09497607 - SecurityId: Other Item: crsp_90387 Signal: -0.010919844 - SecurityId: Other Item: crsp_92464 Signal: 0.0102348536 - SecurityId: Other Item: crsp_90756 Signal: 0.0955377483 - SecurityId: Other Item: crsp_13812 Signal: -0.094404024 - SecurityId: Other Item: crsp_14448 Signal: -0.334669696 - SecurityId: Other Item: crsp_29058 Signal: 0.0160105202 - SecurityId: Other Item: crsp_90995 Signal: -0.001246808 - SecurityId: Other Item: crsp_87696 Signal: 0.000842757 - SecurityId: Other Item: crsp_84827 Signal: -0.015086308 - SecurityId: Other Item: crsp_80329 Signal: 0.0486850872 - SecurityId: Other Item: crsp_30593 Signal: -0.013750603 ... (more)


Get the market capitalization



In [21]:
msfBySecurityIdAndMonth
|> Map.toList
|> List.take 4
|> List.map (fun ((id, month), row) -> id, row.MarketEquity)


index Item1 Item2 0 Item comp_001034_01 Value 1547.12108 1 Item comp_001043_01 Value 3.56 2 Item comp_001076_02 Value 1433.6882 3 Item comp_001081_01 Value 994.7955

In [22]:
let getMarketCap (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> None
    | Some row -> 
        match row.MarketEquity with
        | None -> None
        | Some me -> Some (security, me)



something 



In [23]:
let getSecurityReturn (security, formationMonth) =
    let missingReturn = 0.0
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> security, missingReturn
    | Some x ->  
        match x.Ret with 
        | None -> security, missingReturn
        | Some r -> security, r

Restrictions



In [24]:
let isObsMain (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.ObsMain

let isPrimarySecurity (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.PrimarySec

let isCommonStock (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.Common

let isExchMain (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.ExchMain

let hasMarketEquity (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.MarketEquity.IsSome

let myFilters securityAndFormationMonth =
    isObsMain securityAndFormationMonth &&
    isPrimarySecurity securityAndFormationMonth &&
    isCommonStock securityAndFormationMonth &&
    isExchMain securityAndFormationMonth &&
    isExchMain securityAndFormationMonth &&
    hasMarketEquity securityAndFormationMonth

let doMyFilters (universe:InvestmentUniverse) =
    let filtered = 
        universe.Securities
        // my filters expect security, formationMonth
        |> List.map(fun security -> security, universe.FormationMonth)
        // do the filters
        |> List.filter myFilters
        // now convert back from security, formationMonth -> security
        |> List.map fst
    { universe with Securities = filtered }


Define sample months

In [25]:
let startSample = 
    idAndReturnsRows
    |> List.map(fun row -> DateTime(row.Eom.Year,row.Eom.Month,1))
    |> List.min

let endSample = 
    let lastMonthWithData = 
        idAndReturnsRows
        |> Seq.map(fun row -> DateTime(row.Eom.Year,row.Eom.Month,1))
        |> Seq.max
    lastMonthWithData.AddMonths(-1) 

let sampleMonths = getSampleMonths (startSample, endSample)


Form the strategy

In [26]:
let formStrategy ym =
    ym
    |> getInvestmentUniverse 
    |> doMyFilters
    |> getMySignals
    |> assignSignalSort strategyName 3 
    |> Seq.toList
    |> List.map (giveValueWeights getMarketCap)
    |> List.map (getPortfolioReturn getSecurityReturn)  


In [27]:
let doParallel = true
let portfolios =
    if doParallel then
        sampleMonths
        |> List.toArray
        |> Array.Parallel.map formStrategy
        |> Array.toList
        |> List.collect id
    else
        sampleMonths
        |> List.collect formStrategy


PLOTTING RETURNS


In [28]:
open NovaSBE.Finance.French

let ff3 = getFF3 Frequency.Monthly
let monthlyRiskFreeRate =
    [ for obs in ff3 do 
        let key = DateTime(obs.Date.Year,obs.Date.Month,1)
        key, obs.Rf ]
    |> Map


In [29]:
let portfolioExcessReturns =
    portfolios
    |> List.map(fun x -> 
        match Map.tryFind x.Month monthlyRiskFreeRate with 
        | None -> failwith $"Can't find risk-free rate for {x.Month}"
        | Some rf -> { x with Return = x.Return - rf })


Plot the Long strategy



In [30]:
//Long strategy
let longStrategy = 
    portfolioExcessReturns 
    |> List.filter(fun x -> x.Index = 3)


In [31]:
//Plot the long strategy
let cumulateSimpleReturn (xs: PortfolioReturn list) =
    let xs = xs |> List.sortBy (fun x -> x.Month)
    let mutable cr = 1.0
    [ for x in xs do 
        cr <- cr * (1.0 + x.Return)
        { x with Return = cr - 1.0 } ]

let longCumulative = longStrategy |> cumulateSimpleReturn

let longCumulativeChartLong =
    longCumulative
    |> List.map(fun x -> x.Month, x.Return)
    |> Chart.Line 
    |> Chart.withTitle "Cumulative returns Long Strategy"


In [32]:
longCumulativeChartLong


<!-- Plotly chart will be drawn inside this DIV -->

In [33]:
//Short strategy
let shortStrategy = 
    portfolioExcessReturns 
    |> List.filter(fun x -> 
        x.Name = strategyName && x.Index = 1)

In [34]:
//Plot the short strategy
let shortCumulative = shortStrategy |> cumulateSimpleReturn

let shortCumulativeChart =
    shortCumulative
    |> List.map(fun x -> x.Month, x.Return)
    |> Chart.Line 
    |> Chart.withTitle "Cumulative returns short strategy"


In [35]:
shortCumulativeChart

<!-- Plotly chart will be drawn inside this DIV -->

In [36]:
//Long-short strategy
let longshortStrategy = 
    let shortByYearMonthMap = 
        shortStrategy
        |> List.map(fun row -> row.Month, row) 
        |> Map
    
    [ for longObs in longStrategy do
        match Map.tryFind longObs.Month shortByYearMonthMap with
        | None -> failwith "probably your date variables are not aligned for a weird reason"
        | Some shortObs ->
            { Name = "Long-Short"
              Index = 1
              Month = longObs.Month
              Return = longObs.Return - shortObs.Return } ]

In [37]:
//Plot the longshort strategy
let longshortCumulative = longshortStrategy |> cumulateSimpleReturn

let longshortCumulativeChart =
    longshortCumulative
    |> List.map(fun x -> x.Month, x.Return)
    |> Chart.Line 
    |> Chart.withTitle "Cumulative returns long short strategy"

In [38]:
longshortCumulativeChart

<!-- Plotly chart will be drawn inside this DIV -->

Multiple portfolio plot


In [39]:
let vwMktRf =
    let portfolioMonths = 
        portfolioExcessReturns 
        |> List.map(fun x -> x.Month)
    let minYm = portfolioMonths |> List.min
    let maxYm = portfolioMonths |> List.max
    
    [ for x in ff3 do
        if x.Date >= minYm && x.Date <= maxYm then
            { Name = "Mkt-Rf"
              Index = 1
              Month = x.Date
              Return = x.MktRf } ]


A function to do the plot

In [40]:
let portfolioReturnPlot (xs:PortfolioReturn list) =
    xs
    |> List.map(fun x -> x.Month, x.Return)
    |> Chart.Line 
    |> Chart.withTitle "Growth of 1 Euro" 

In [41]:

let combinedChart =
    List.concat [longStrategy; longshortStrategy; vwMktRf]
    |> List.groupBy(fun x -> x.Name, x.Index)
    |> List.map(fun ((name, index), xs) ->
        xs
        |> cumulateSimpleReturn
        |> portfolioReturnPlot
        |> Chart.withTraceInfo (Name=($"{name}: {index}")))
    |> Chart.combine



In [42]:
combinedChart

<!-- Plotly chart will be drawn inside this DIV -->

1) A function to annualize the monthly stdev
2) Calculate stdev

In [43]:
let annualizeMonthlyStdDev monthlyStdDev = sqrt(12.0) * monthlyStdDev

let stdDevLongAnnualized =
    longStrategy
    |> Seq.stDevBy (fun x -> x.Return)
    |> annualizeMonthlyStdDev

let stdDevLongShortAnnualized =
    longshortStrategy
    |> Seq.stDevBy (fun x -> x.Return)
    |> annualizeMonthlyStdDev

let stdDevVwMktRfAnnualized =
    vwMktRf
    |> Seq.stDevBy (fun x -> x.Return)
    |> annualizeMonthlyStdDev

Calculate the leverage to apply to each portfolio, so that they all have an annualized volatility of 10% over the full sample

In [44]:
let leverageLong = 0.1 / stdDevLongAnnualized
let leverageLongShort = 0.1 / stdDevLongShortAnnualized
let leverageVwMktRf = 0.1/ stdDevVwMktRfAnnualized

Apply the leverage

In [45]:

let long10 = 
    longStrategy
    |> List.map(fun (x) ->
        { Name = x.Name;
          Index = x.Index
          Month= x.Month;
          Return = leverageLong * x.Return })

let longShort10 = 
    longshortStrategy
    |> List.map(fun (x) ->
        { Name = x.Name;
          Index = x.Index
          Month= x.Month;
          Return = leverageLongShort * x.Return })

let vwMktRf10 = 
  vwMktRf
    |> List.map(fun (x) ->
        { Name = x.Name;
          Index = x.Index
          Month= x.Month;
          Return = leverageVwMktRf * x.Return })

Plot che portfolios with the leverage

In [46]:
let combinedChart10 =
    List.concat [long10; longShort10; vwMktRf10]
    |> List.groupBy(fun x -> x.Name, x.Index)
    |> List.map(fun ((name, index), xs) ->
        xs
        |> cumulateSimpleReturn
        |> portfolioReturnPlot
        |> Chart.withTraceInfo (Name=($"{name}: {index}"))) 
    |> Chart.combine

In [47]:
combinedChart10


<!-- Plotly chart will be drawn inside this DIV -->

2 function to divide the entire portfolio in 2 halves.
First half from 2000 to 2010
Second half from 2011 to 2020

In [48]:


let filterBeforeDate (returns: PortfolioReturn list) =
    let targetDate = DateTime(2010, 12, 2)
    List.filter (fun ret -> ret.Month < targetDate) returns

let filterAfterDate (returns: PortfolioReturn list) =
    let targetDate = DateTime(2010, 12, 2)
    List.filter (fun ret -> ret.Month > targetDate) returns


Dividing the 3 strategies using the function above

In [49]:

let LongStrategy1Half = filterBeforeDate longStrategy
let LongStrategy2Half = filterAfterDate longStrategy

let LongShortStrategy1Half = filterBeforeDate longshortStrategy
let LongShortStrategy2Half = filterAfterDate longshortStrategy

let vwMktRf1Half = filterBeforeDate vwMktRf
let vwMktRf2Half = filterAfterDate vwMktRf


Summary statistics 

A function to calculate the Annualized Return giving as an input a list of portfolio return

In [50]:

let calculateAverageAnnualizedReturn (returns: PortfolioReturn list) =
    returns
    |> List.map(fun x -> x.Return)
    |> List.average
    |> fun x -> (1.0 + x)**12.0 - 1.0

    


Using the function above

In [51]:
//for the long strategy
let annAverRetulongStrategyFullSample = calculateAverageAnnualizedReturn longStrategy
let annAverRetulongStrategy1half = calculateAverageAnnualizedReturn LongStrategy1Half
let annAverRetulongStrategy2half = calculateAverageAnnualizedReturn LongStrategy2Half

//for the long-short strategy
let annAverRetulongshortStrategyFullSample = calculateAverageAnnualizedReturn longshortStrategy
let annAverRetulongshortStrategy1half = calculateAverageAnnualizedReturn LongShortStrategy1Half
let annAverRetulongshortStrategy2half = calculateAverageAnnualizedReturn LongShortStrategy2Half

//for the value weighted market Portfolio
let annAverRetuvwMktRfFullSample = calculateAverageAnnualizedReturn vwMktRf
let annAverRetuvwMktRf1half = calculateAverageAnnualizedReturn vwMktRf1Half
let annAverRetuvwMktRf2half = calculateAverageAnnualizedReturn vwMktRf2Half


annAverRetulongshortStrategy1half

0.0626650312182242

Function to calculate the standard deviation

In [52]:
// function to calculate the standard deviation
let AnnualizedStdev (returns: PortfolioReturn list) =
    returns
    |> List.map(fun x -> x.Return)
    |> stDev
    |> fun x -> x*sqrt(12.0)



Using the function above

In [53]:
//for the long strategy
let StDevlongStrategyFullSample = AnnualizedStdev longStrategy
let StDevlongStrategy1half = AnnualizedStdev LongStrategy1Half
let StDevlongStrategy2half = AnnualizedStdev LongStrategy2Half

//for the long-short strategy
let StDevlongshortStrategyFullSample = AnnualizedStdev longshortStrategy
let StDevlongshortStrategy1half = AnnualizedStdev LongShortStrategy1Half
let StDevlongshortStrategy2half = AnnualizedStdev LongShortStrategy2Half

//for the value weighted market Portfolio
let StDevvwMktRfFullSample = AnnualizedStdev vwMktRf
let StDevvwMktRf1half = AnnualizedStdev vwMktRf1Half
let StDevvwMktRf2half = AnnualizedStdev vwMktRf2Half


A function to calculate the sharpe ratio

In [54]:
//a function to calculate the sharpe ratio
let calculateSharpeRatio (annualizedReturn: float) (annualizedStdDev: float) =
    let sharpeRatio = annualizedReturn / annualizedStdDev
    sharpeRatio



Using the function above

In [55]:
//for the long strategy
let SharpeRatiolongStrategyFullSample = calculateSharpeRatio annAverRetulongStrategyFullSample StDevlongStrategyFullSample
let SharpeRatiolongStrategy1half = calculateSharpeRatio annAverRetulongStrategy1half StDevlongStrategy1half
let SharpeRatiolongStrategy2half = calculateSharpeRatio annAverRetulongStrategy2half StDevlongStrategy2half

//for the long-short strategy
let SharpeRatiolongshortStrategyFullSample = calculateSharpeRatio annAverRetulongshortStrategyFullSample StDevlongshortStrategyFullSample
let SharpeRatiolongshortStrategy1half = calculateSharpeRatio annAverRetulongshortStrategy1half StDevlongshortStrategy1half
let SharpeRatiolongshortStrategy2half = calculateSharpeRatio annAverRetulongshortStrategy2half StDevlongshortStrategy2half

//for the value weighted market Portfolio
let SharpeRatiovwMktRfFullSample = calculateSharpeRatio annAverRetuvwMktRfFullSample StDevvwMktRfFullSample
let SharpeRatiovwMktRf1half = calculateSharpeRatio annAverRetuvwMktRf1half StDevvwMktRf1half
let SharpeRatiovwMktRf2half = calculateSharpeRatio annAverRetuvwMktRf2half StDevvwMktRf2half

SharpeRatiovwMktRf2half

1.028284565678377

In [56]:
#r "nuget: Accord"
#r "nuget: Accord.Statistics"

open Accord
open Accord.Statistics.Models.Regression.Linear


type RegData =
    { Date : DateTime
      Portfolio : float
      MktRf : float 
      Hml : float 
      Smb : float }


let ff3ByMonth = 
    ff3
    |> Seq.map(fun x -> DateTime(x.Date.Year, x.Date.Month,1), x)
    |> Map

Installed Packages Accord, 3.8.0 Accord.Statistics, 3.8.0

Transforming all the strategies in type RegData

In [57]:
//for my long only portfolio
let longRegData =
    longStrategy
    |> List.map(fun port ->
        let monthToFind = DateTime(port.Month.Year,port.Month.Month,1)
        match Map.tryFind monthToFind ff3ByMonth with
        | None -> failwith "probably you messed up your days of months"
        | Some ff3 -> 
            { Date = monthToFind
              Portfolio = port.Return 
              MktRf = ff3.MktRf 
              Hml = ff3.Hml 
              Smb = ff3.Smb })
    |> List.toArray

let long1stRegData =
    LongStrategy1Half
    |> List.map(fun port ->
        let monthToFind = DateTime(port.Month.Year,port.Month.Month,1)
        match Map.tryFind monthToFind ff3ByMonth with
        | None -> failwith "probably you messed up your days of months"
        | Some ff3 -> 
            { Date = monthToFind
              Portfolio = port.Return 
              MktRf = ff3.MktRf 
              Hml = ff3.Hml 
              Smb = ff3.Smb })
    |> List.toArray

let long2ndRegData =
    LongStrategy2Half
    |> List.map(fun port ->
        let monthToFind = DateTime(port.Month.Year,port.Month.Month,1)
        match Map.tryFind monthToFind ff3ByMonth with
        | None -> failwith "probably you messed up your days of months"
        | Some ff3 -> 
            { Date = monthToFind
              Portfolio = port.Return 
              MktRf = ff3.MktRf 
              Hml = ff3.Hml 
              Smb = ff3.Smb })
    |> List.toArray



In [58]:
//for my long-short portfolio
let longShortRegData =
    longshortStrategy
    |> List.map(fun port ->
        let monthToFind = DateTime(port.Month.Year,port.Month.Month,1)
        match Map.tryFind monthToFind ff3ByMonth with
        | None -> failwith "probably you messed up your days of months"
        | Some ff3 -> 
            { Date = monthToFind
              Portfolio = port.Return
              MktRf = ff3.MktRf 
              Hml = ff3.Hml 
              Smb = ff3.Smb })
    |> List.toArray

let longShort1stRegData =
    LongShortStrategy1Half
    |> List.map(fun port ->
        let monthToFind = DateTime(port.Month.Year,port.Month.Month,1)
        match Map.tryFind monthToFind ff3ByMonth with
        | None -> failwith "probably you messed up your days of months"
        | Some ff3 -> 
            { Date = monthToFind
              Portfolio = port.Return 
              MktRf = ff3.MktRf 
              Hml = ff3.Hml 
              Smb = ff3.Smb })
    |> List.toArray

let longShort2ndRegData =
    LongShortStrategy2Half
    |> List.map(fun port ->
        let monthToFind = DateTime(port.Month.Year,port.Month.Month,1)
        match Map.tryFind monthToFind ff3ByMonth with
        | None -> failwith "probably you messed up your days of months"
        | Some ff3 -> 
            { Date = monthToFind
              Portfolio = port.Return 
              MktRf = ff3.MktRf 
              Hml = ff3.Hml 
              Smb = ff3.Smb })
    |> List.toArray

Calculate the CAPM alpha and t statistic

In [59]:
//fit CAPM and fama french 3 factors model for my long portfolios
open NovaSBE.Finance.Ols
//full sample
let capmModellong = Ols("Portfolio ~ MktRf", longRegData).fit()
let ff3Modellong = Ols("Portfolio ~ MktRf + Hml + Smb", longRegData).fit()

//1 half of the sample
let capmModellong1half = Ols("Portfolio ~ MktRf", long1stRegData).fit()
let ff3Modellong1half = Ols("Portfolio ~ MktRf + Hml + Smb", long1stRegData).fit()

//2 half of the sample
let capmModellong2half = Ols("Portfolio ~ MktRf", long2ndRegData).fit()
let ff3Modellong2half = Ols("Portfolio ~ MktRf + Hml + Smb", long2ndRegData).fit()

In [60]:
//fit CAPM and fama french 3 factors model for my long-short portfolios
//full sample
let capmModellongshort = Ols("Portfolio ~ MktRf", longShortRegData).fit()
let ff3Modellongshort = Ols("Portfolio ~ MktRf + Hml + Smb", longShortRegData).fit()

//1 half of the sample
let capmModellongshort1half = Ols("Portfolio ~ MktRf", longShort1stRegData).fit()
let ff3Modellongshort1half = Ols("Portfolio ~ MktRf + Hml + Smb", longShort1stRegData).fit()

//2 half of the sample
let capmModellongshort2half = Ols("Portfolio ~ MktRf", longShort2ndRegData).fit()
let ff3Modellongshort2half = Ols("Portfolio ~ MktRf + Hml + Smb", longShort2ndRegData).fit()

In [61]:
//Find the CAPM alpha and t statistic for my long strategy
//fullsample
let capmAlphalong =  capmModellong.coefs["Intercept"]
let capmAlphaTlong = capmModellong.tvalues["Intercept"]
//1half
let capmAlphalong1half = capmModellong1half.coefs["Intercept"]
let capmAlphaTlong1half = capmModellong1half.tvalues["Intercept"]
//2half
let capmAlphalong2half = capmModellong2half.coefs["Intercept"]
let capmAlphaTlong2half = capmModellong2half.tvalues["Intercept"]




In [62]:
//Find the CAPM alpha and t statistic for my long-short strategy
//fullsample
let capmAlphalongshort =  capmModellongshort.coefs["Intercept"]
let capmAlphaTlongshort = capmModellongshort.tvalues["Intercept"]
//1half
let capmAlphalongshort1half = capmModellongshort1half.coefs["Intercept"]
let capmAlphaTlongshort1half = capmModellongshort1half.tvalues["Intercept"]
//2half
let capmAlphalongshort2half = capmModellongshort2half.coefs["Intercept"]
let capmAlphaTlongshort2half = capmModellongshort2half.tvalues["Intercept"]



Finding the information ratio

In [63]:
let informationRatio monthlyAlpha (monthlyResiduals: float array) =
    monthlyAlpha / (Seq.stDev monthlyResiduals)


In [64]:
//residuals long strategy
//full sample
let capmAlphalongResiduals =  capmModellong.resid
//1half
let capmAlphalong1halfResiduals = capmModellong1half.resid
//2half
let capmAlphalong2halfResiduals = capmModellong2half.resid

//full sample IR
let capmIRlong =  informationRatio capmAlphalong capmAlphalongResiduals
//1half
let capmIRlong1half = informationRatio capmAlphalong1half capmAlphalong1halfResiduals
//2half
let capmIRlong2half = informationRatio capmAlphalong2half capmAlphalong2halfResiduals




In [65]:

//residuals long-short strategy
//full sample
let capmAlphalongshortResiduals =  capmModellongshort.resid
//1half
let capmAlphalongshort1halfResiduals = capmModellongshort1half.resid
//2half
let capmAlphalongshort2halfResiduals = capmModellongshort2half.resid



//full sample IR
let capmIRlongshort =  informationRatio capmAlphalongshort capmAlphalongshortResiduals
//1half
let capmIRlongshort1half = informationRatio capmAlphalongshort1half capmAlphalongshort1halfResiduals
//2half
let capmIRlongshort2half = informationRatio capmAlphalongshort2half capmAlphalongshort2halfResiduals

capmIRlongshort2half

0.000953978178634025

# STRATEGY AS PART OF DIVERSIFIED PORTFOLIO

In [66]:

#r "nuget: NovaSBE.Finance"
open FSharp.Data
open Quotes.YahooFinance

Installed Packages NovaSBE.Finance, 0.3.0-beta.2

In [67]:
#r "nuget: DiffSharp-lite"
open DiffSharp

Installed Packages diffsharp-lite, 1.0.7

In [68]:
Formatter.Register<Tensor>(
    Action<_, _>(fun t (writer: IO.TextWriter) -> fprintfn writer "%120A" t),"text/plain")
Formatter.Register<Tensor>(
    Action<_, _>(fun t (writer: IO.TextWriter) -> fprintfn writer "%120A" t),"text/html")
Formatter.SetPreferredMimeTypesFor(typeof<Tensor>, "text/plain")

In [69]:
type StockData =
    { Symbol : string 
      Date : DateTime
      Return : float }

In [70]:
let ff3a = ff3 |> Array.toList
let ff3StockData =
    [ 
       ff3a |> List.map(fun x -> {Symbol="HML";Date=x.Date;Return=x.Hml})
       ff3a |> List.map(fun x -> {Symbol="MktRf";Date=x.Date;Return=x.MktRf})
       ff3a |> List.map(fun x -> {Symbol="Smb";Date=x.Date;Return=x.Smb})
    ] |> List.concat

In [71]:
let tickers = 
    [ 
        "VTI" // Vanguard Total Stock Market ETF
        "BND" // Vanguard Total Bond Market ETF
    ]

let tickPrices = 
    YahooFinance.History(
        tickers,
        startDate = DateTime(2010,1,1),
        interval = Interval.Monthly)



In [72]:
let pricesToReturns (symbol, adjPrices: list<Quote>) =
    adjPrices
    |> List.sortBy (fun x -> x.Date)
    |> List.pairwise
    |> List.map (fun (day0, day1) ->
        let r = day1.AdjustedClose / day0.AdjustedClose - 1.0 
        { Symbol = symbol 
          Date = day1.Date 
          Return = r })



In [73]:
let tickReturns =
    tickPrices
    |> List.groupBy (fun x -> x.Symbol)
    |> List.collect pricesToReturns

Transforming my portfolios in StockData type

In [74]:

let LongStrategyEfficient =
    longStrategy
    |> List.map(fun port -> {Symbol = "LongEfficient"; Date = port.Month; Return = port.Return})

let LongshortStrategyEfficient =
    longshortStrategy
    |> List.map(fun port -> {Symbol = "LongShortEfficient"; Date = port.Month; Return = port.Return})

In [75]:
let rf = Map [ for x in ff3a do x.Date, x.Rf ]

let standardInvestmentsExcess =
    let maxff3Date = ff3a |> List.map(fun x -> x.Date) |> List.max
    tickReturns
    |> List.filter(fun x -> x.Date <= maxff3Date)
    |> List.map(fun x -> 
        match Map.tryFind x.Date rf with 
        | None -> failwith $"why isn't there a rf for {x.Date}"
        | Some rf -> { x with Return = x.Return - rf })

let initial_date = tickReturns[0].Date
let maxff3Date = ff3a |> List.map(fun x -> x.Date) |> List.max

In [76]:
let standardInvestmentsExcessLongStrategy=
    LongStrategyEfficient
    |> List.filter(fun x -> x.Date >= initial_date)
    |> List.filter(fun x -> x.Date <= maxff3Date)
    |> List.map(fun x -> 
        match Map.tryFind x.Date rf with 
        | None -> failwith $"why isn't there a rf for {x.Date}"
        | Some rf -> { x with Return = x.Return })

let standardInvestmentsExcessLongshortStrategy=
    LongshortStrategyEfficient
    |> List.filter(fun x -> x.Date >= initial_date)
    |> List.filter(fun x -> x.Date <= maxff3Date)
    |> List.map(fun x -> 
        match Map.tryFind x.Date rf with 
        | None -> failwith $"why isn't there a rf for {x.Date}"
        | Some rf -> { x with Return = x.Return })

Merging my strategies with the ETF's

In [77]:
let standardInvestmentsExcessLongStrategy1 = List.concat[|standardInvestmentsExcess; standardInvestmentsExcessLongStrategy|] |> Seq.distinct |> List.ofSeq |> List.groupBy(fun x -> x.Symbol) |> Map
    
let standardInvestmentsExcessLongshortStrategy1 =  List.concat[|standardInvestmentsExcess; standardInvestmentsExcessLongshortStrategy|] |> Seq.distinct |> List.ofSeq |> List.groupBy(fun x -> x.Symbol) |> Map

Creating list of tickers for each of the strategies

In [78]:
let tickersLongStrategy =
    [ 
        "VTI" // Vanguard Total Stock Market ETF
        "BND" // Vanguard Total Bond Market ETF
        "LongEfficient"
    ]

let tickersLongShortStrategy =
    [ 
        "VTI" // Vanguard Total Stock Market ETF
        "BND" // Vanguard Total Bond Market ETF
        "LongShortEfficient"
    ]

let tickers6040 =
    [ 
        "VTI" // Vanguard Total Stock Market ETF
        "BND" // Vanguard Total Bond Market ETF
    ]

A function to get the covariance, with overlapping dates

In [79]:
//covariance
let getCov xId yId (createPortfolio: Map<string,StockData list>) =
    let xRet = 
        createPortfolio[xId] 
        |> List.map (fun x -> x.Date,x.Return) 
        |> Map
    let yRet = 
        createPortfolio[yId]
        |> List.map (fun y -> y.Date, y.Return)
        |> Map
    let overlappingDates =
        [ xRet.Keys |> set
          yRet.Keys |> set]
        |> Set.intersectMany
    [ for date in overlappingDates do xRet[date], yRet[date]]
    |> Seq.covOfPairs


Using the function above

In [80]:
let CovLongStrategy =
    [ for rowTick in tickersLongStrategy do 
        [ for colTick in tickersLongStrategy do
            getCov rowTick colTick standardInvestmentsExcessLongStrategy1 ]]
    |> dsharp.tensor


In [81]:
let CovLongShortStrategy =
    [ for rowTick in tickersLongShortStrategy do 
        [ for colTick in tickersLongShortStrategy do
            getCov rowTick colTick standardInvestmentsExcessLongshortStrategy1 ]]
    |> dsharp.tensor


Getting the weight for the mean variance efficient portfolio

In [82]:
let AverageLongStrategy =
    [ for ticker in tickersLongStrategy do 
        standardInvestmentsExcessLongStrategy1[ticker]
        |> List.averageBy (fun x -> x.Return)]
    |> dsharp.tensor

let l_w' = dsharp.solve(CovLongStrategy,AverageLongStrategy)
let wLong = l_w' / l_w'.sum()

let weightsLong =
    Seq.zip tickersLongStrategy (wLong.toArray1D<float>())
    |> Map.ofSeq


In [83]:
let AverageLongShortStrategy =
    [ for ticker in tickersLongShortStrategy do 
        standardInvestmentsExcessLongshortStrategy1[ticker]
        |> List.averageBy (fun x -> x.Return)]
    |> dsharp.tensor

let ls_w' = dsharp.solve(CovLongShortStrategy,AverageLongShortStrategy)
let wLongShort = ls_w' / ls_w'.sum()

let weightsLongShort =
    Seq.zip tickersLongShortStrategy (wLongShort.toArray1D<float>())
    |> Map.ofSeq

Sorting by date

In [84]:
let SortedstockDataLong =  standardInvestmentsExcessLongStrategy1.Values |> Seq.toList |> List.collect id |> List.groupBy(fun x -> x.Date) |> List.sortBy fst
let SortedstockDataLongShort = standardInvestmentsExcessLongshortStrategy1.Values |> Seq.toList |> List.collect id |> List.groupBy(fun x -> x.Date) |> List.sortBy fst

Finding the first and the last month where i have all the 3 assets

In [85]:
let allAssetsStartLong =
    SortedstockDataLong
    |> List.find(fun (month, stocks) -> stocks.Length = tickersLongStrategy.Length)
    |> fst 

let allAssetsEndLong =
    SortedstockDataLong
    |> List.findBack(fun (month, stocks) -> stocks.Length = tickersLongStrategy.Length)
    |> fst

let allAssetsStartLongShort =
    SortedstockDataLongShort
    |> List.find(fun (month, stocks) -> stocks.Length = tickersLongShortStrategy.Length)
    |> fst 

let allAssetsEndLongShort =
    SortedstockDataLongShort
    |> List.findBack(fun (month, stocks) -> stocks.Length = tickersLongShortStrategy.Length)
    |> fst

let stockDataByDateCompleteLong =
    SortedstockDataLong
    |> List.filter(fun (date, stocks) -> 
        date >= allAssetsStartLong &&
        date <= allAssetsEndLong)

let stockDataByDateCompleteLongShort =
    SortedstockDataLongShort
    |> List.filter(fun (date, stocks) -> 
        date >= allAssetsStartLongShort &&
        date <= allAssetsEndLongShort)

Check that is working

In [86]:
let checkOfCompleteDataLong =
    stockDataByDateCompleteLong
    |> List.map snd
    |> List.filter(fun x -> x.Length <> tickersLongStrategy.Length) // discard rows where we have all symbols.

if not (List.isEmpty checkOfCompleteDataLong) then 
        failwith "stockDataByDateComplete has months with missing stocks"


let checkOfCompleteData_longShort =
    stockDataByDateCompleteLongShort
    |> List.map snd
    |> List.filter(fun x -> x.Length <> tickersLongShortStrategy.Length) // discard rows where we have all symbols.

if not (List.isEmpty checkOfCompleteData_longShort) then 
        failwith "stockDataByDateComplete has months with missing stocks"

A function to calculate the return based on the weight

In [87]:
let portfolioMonthReturn (weights: Map<string,float>) (monthData: list<StockData>) =
    weights
    |> Map.toList
    |> List.map(fun (symbol, weight) ->
        let symbolData = 
            match monthData |> List.tryFind(fun x -> x.Symbol = symbol) with
            | None -> failwith $"You tried to find {symbol} in the data but it was not there"
            | Some data -> data
        symbolData.Return*weight)
    |> List.sum

In [88]:
let weights6040 = Map [("VTI",0.6);("BND",0.4)]


Creating my actual data to work on for each portfolio

In [89]:
let MVElong =
    stockDataByDateCompleteLong
    |> List.map(fun (date, data) -> 
        { Symbol = "MVE Long Portfolio"
          Date = date
          Return = portfolioMonthReturn weightsLong data })

let MVElongShort =
    stockDataByDateCompleteLongShort
    |> List.map(fun (date, data) -> 
        { Symbol = "MVE Long-Short"
          Date = date
          Return = portfolioMonthReturn weightsLongShort data })


let MVE6040 = 
    stockDataByDateCompleteLong
    |> List.map(fun (date, data) -> 
        { Symbol = "60/40 Portfolio"
          Date = date 
          Return = portfolioMonthReturn weights6040 data})


1) A function to calculate the cumulative returns 
2) Then a function that get as input the strategies, calculate the cumulative returns and then plot it


In [90]:
let cumulateReturns (xs: list<StockData>) =
    let mutable cr = 1.0
    [ for x in xs do 
        cr <- cr * (1.0 + x.Return)
        { x with Return = cr } ]

let Plot3Strategies (first: list<StockData>) (second: list<StockData>) (third: list<StockData>) =
     let MveLongCumulative = first |> cumulateReturns |> List.map(fun x -> x.Date, x.Return) |> Chart.Line |> Chart.withTraceInfo(Name="MVE Long")
     let MveLongShortCumulative = second |> cumulateReturns |> List.map(fun x -> x.Date, x.Return) |> Chart.Line |> Chart.withTraceInfo(Name="MVE Long Short")
     let MVE6040Cumulative = third |> cumulateReturns |> List.map(fun x -> x.Date, x.Return)  |> Chart.Line |> Chart.withTraceInfo(Name="60/40")
     let chartCombined = [ MveLongCumulative; MVE6040Cumulative;MveLongShortCumulative] |> Chart.combine|> Chart.withTitle("Diversified Portfolio - Cumulative Returns")
     chartCombined
     
Plot3Strategies MVElong MVElongShort MVE6040


<!-- Plotly chart will be drawn inside this DIV -->

Now with leverage
1) A function to calculate the leverage
2) Same function as before but with leverage

In [91]:


let Leverage10 portfolio =
    let vol = portfolio |> List.map(fun x -> x.Return) |> Seq.stDev
    let annualizedVol = vol * sqrt(12.0)
    portfolio 
    |> List.map(fun x -> { x with Return = x.Return * (0.1/annualizedVol)})

let Plot3Strategies10leverage (first: list<StockData>) (second: list<StockData>) (third: list<StockData>) =
     let LevCumulativelong = first |> Leverage10 |> cumulateReturns |> List.map(fun x -> x.Date, x.Return) |> Chart.Line |> Chart.withTraceInfo(Name="MVE Long")
     let LevCumulativelongshort = second |> Leverage10 |> cumulateReturns |> List.map(fun x -> x.Date, x.Return) |> Chart.Line |> Chart.withTraceInfo(Name="MVE Long Short")
     let LevCumulativelongshort6040 = third |> Leverage10 |> cumulateReturns |> List.map(fun x -> x.Date, x.Return)  |> Chart.Line |> Chart.withTraceInfo(Name="60/40")
     let chartCombined = [ LevCumulativelong; LevCumulativelongshort6040;LevCumulativelongshort] |> Chart.combine|> Chart.withTitle("Diversified Portfolio - Cumulative Returns")
     chartCombined

Plot3Strategies10leverage MVElong MVElongShort MVE6040


<!-- Plotly chart will be drawn inside this DIV -->

Summary statistics

Calculate annualized average excess returns for each portfolio

In [92]:
let AnnualizedExcessReturns (strategy: list<StockData>) =
    strategy
    |> List.map(fun x -> x.Return)
    |> List.map(fun x -> x*12.0)
    |> List.average 

let AVGDataLong = AnnualizedExcessReturns MVElong
let AVGDataLongShort  = AnnualizedExcessReturns MVElongShort
let AVGData6040 = AnnualizedExcessReturns MVE6040

   

Now the sharpe ratio

In [93]:

let Sharpe (xs: float seq) =
    (Seq.mean xs) / (Seq.stDev xs)


let SharpeLong = 
    MVElong
    |> List.map (fun x -> x.Return )
    |> Sharpe
    |> fun x -> x*sqrt(12.0)

let SharpeLongShort = 
    MVElongShort
    |> List.map (fun x -> x.Return)
    |> Sharpe
    |> fun x -> x*sqrt(12.0)

let Sharpe6040 = 
    MVE6040
    |> List.map (fun x -> x.Return)
    |> Sharpe
    |> fun x -> x*sqrt(12.0)

Same thing for this strategy, but applying the leverage 

In [94]:
let AnnualizeSharpeRatio (strategy: list<StockData>) =
    strategy 
    |> List.map (fun x -> x.Return)
    |> Sharpe
    |> fun x -> x*sqrt(12.0)

let AVGlong10 = MVElong |> Leverage10 |> AnnualizedExcessReturns 
let AVGLongShort10  = MVElongShort |> Leverage10 |> AnnualizedExcessReturns 
let AVG604010 = MVE6040 |> Leverage10 |> AnnualizedExcessReturns 



let SharpeLong10 = MVElong |> Leverage10 |> AnnualizeSharpeRatio
let SharpeLongShort10 = MVElongShort |> Leverage10 |> AnnualizeSharpeRatio
let Sharpe604010 = MVE6040 |> Leverage10 |> AnnualizeSharpeRatio

